In [106]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from pprint import pprint
import sys

def spotipy_authorize(cid: str, secret: str):
    client_credentials_manager = SpotifyClientCredentials(cid, secret)
    return spotipy.Spotify(client_credentials_manager)

sp = spotipy.Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id="da82a340bd1341599bd3c590de6ad9fa", 
        client_secret="07c1f564fad9402da3fecc0fa65c0f7b"
    )
)

playlist_link = 'https://open.spotify.com/playlist/4iuLi6QYYPPFV4QHCZB7iU?si=1a043c5580ed416d'
# playlist_link = 'https://open.spotify.com/playlist/37i9dQZF1DZ06evO2HOPhS?si=a0b46fd429fb4b2a'
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

In [103]:

def call_playlist(playlist_uri : str, creator='spotify') -> pd.DataFrame:
    

    playlist_features_list = ['artist', 'album', 'track_name', 'track_id','url', 'artist_popularity', 'artist_genre', 'danceability',
                                'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness','instrumentalness', 'liveness', 
                                'valence', 'tempo', 'duration_ms', 'time_signature']
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    artist_df = pd.DataFrame(columns=['artist_popularity', 'artist_genre'])
    
    playlist = sp.user_playlist_tracks(creator, playlist_uri)["items"]
    for track in playlist:
        playlist_features = {}
        artist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        playlist_features["url"] = track['track']['album']['images'][1]['url']
        
        artist_uri = track["track"]["artists"][0]["uri"]
        artist_info = sp.artist(artist_uri)
        
        
        artist_features["artist_popularity"] = artist_info["popularity"]
        artist_features["artist_genre"] = str(artist_info["genres"])
        popularity = (artist_info["popularity"])
        genres = str(artist_info["genres"])
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[7:]:
            playlist_features[feature] = audio_features[feature]
            
        # Concat the 
        artist_df = pd.DataFrame(artist_features, index=[0])
        track_df = pd.DataFrame(playlist_features, index = [0])
        
        artist_track_df = pd.concat([track_df, artist_df], axis=1)
        
        playlist_df = pd.concat([playlist_df, artist_track_df], ignore_index = True)
            
    return playlist_df

In [104]:
df = call_playlist(playlist_URI)

In [105]:
df.head()

,artist,album,track_name,track_id,url,artist_popularity,artist_genre,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,León Larregui,Voluma,Locos,3GSMdtJphymHEsR8K9jT5Q,https://i.scdn.co/image/ab67616d00001e022a05b1...,68,"['latin alternative', 'latin rock', 'rock en e...",0.724,0.646,10,-6.475,1,0.0352,0.0720,0.000157,0.0923,0.957,112.019,177600,4
1,León Larregui,Solstis - De León Larregui,Brillas,0SRddBTphQwQcfqw4Br1uX,https://i.scdn.co/image/ab67616d00001e025be353...,68,"['latin alternative', 'latin rock', 'rock en e...",0.625,0.433,9,-12.963,1,0.0366,0.3760,0.001030,0.2480,0.274,93.041,226187,4
2,León Larregui,Metrópolis (Live),Rue Vieille Du Temple - Live,3a9KZagWgkTU2YSsCtlMRl,https://i.scdn.co/image/ab67616d00001e027f95db...,68,"['latin alternative', 'latin rock', 'rock en e...",0.545,0.789,9,-5.164,0,0.0262,0.1200,0.000024,0.7520,0.663,149.977,239000,4
3,Bunbury,MTV Unplugged. El Libro De Las Mutaciones,La chispa adecuada (feat. León Larregui) - MTV...,5vnAu12wplLvx1XH01PwRH,https://i.scdn.co/image/ab67616d00001e02c4b256...,69,"['latin alternative', 'latin rock', 'rock en e...",0.521,0.482,0,-9.083,0,0.0291,0.1530,0.000000,0.7500,0.307,115.997,291117,4
4,León Larregui,Voluma,Birdie,1BokDaNfW9LVbEHBABNIUv,https://i.scdn.co/image/ab67616d00001e022a05b1...,68,"['latin alternative', 'latin rock', 'rock en e...",0.602,0.521,5,-9.963,1,0.0330,0.0864,0.393000,0.1110,0.631,90.023,197160,4


In [7]:
df.to_csv('testing-data.csv', index=False)

In [107]:
from skimage import  io
import matplotlib.pyplot as plt

def visualize_songs(df):
    """ 
    Visualize cover art of the songs in the inputted dataframe

    Parameters: 
        df (pandas dataframe): Playlist Dataframe
    """
    
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(len(temp) / columns + 1, columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks(color = 'w', fontsize = 0.1)
        plt.yticks(color = 'w', fontsize = 0.1)
        plt.xlabel(df['name'].values[i], fontsize = 12)
        plt.tight_layout(h_pad=0.4, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()

In [108]:
from skimage import io
import matplotlib.pyplot as plt

def visualize_cover_art(playlist_df):
    temp = playlist_df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    
    for i, url in enumerate(temp):
        plt.subplot(int(len(temp) / columns + 1), columns, i + 1)

        image = io.imread(url)
        plt.imshow(image)
        plt.xticks([])
        plt.yticks([])
        s='' 
        plt.xlabel(s.join(playlist_df['track_name'].values[i].split(' ')[:4]), fontsize = 10, fontweight='bold')
        plt.tight_layout(h_pad=0.8, w_pad=0)
        plt.subplots_adjust(wspace=None, hspace=None)

    plt.show()